# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_Copy1 import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
weather_data = pd.read_csv (r'../WeatherPy/weather_data.csv')
weather_data.head()

,City,Country,Date,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Mar del Plata,AR,1605322800,-38.0023,-57.5575,63.75,90,49,13.85
1,Butaritari,KI,1605322800,3.0707,172.7902,81.68,67,69,8.70
2,Flinders,AU,1605322800,-34.5833,150.8552,74.25,57,0,7.29
3,Qaanaaq,GL,1605322800,77.4840,-69.3632,1.69,76,71,5.99
4,Port Elizabeth,ZA,1605322800,-33.9180,25.5701,64.44,82,0,12.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
#store lat and long in locations
locations=weather_data[["Lat","Lng"]]
#set plot info
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
hum_fig=gmaps.figure(layout=figure_layout)
humidity=weather_data["Humidity"]
markers=gmaps.marker_layer(locations)
hum_fig.add_layer(markers)
#plot


In [25]:
gmaps.configure(api_key=g_key)
hum_fig=gmaps.figure(map_type='HYBRID')
heatmap_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
hum_fig.add_layer(heatmap_layer)
hum_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
#narrow down weather_data for idea conditions:temp 70-80 degrees, windspeed<10, 0 cloudiness
cond1=weather_data["Max Temperature"]>=70
cond2=weather_data["Max Temperature"]<=80
cond3=weather_data["Wind Speed"]<10
cond4=weather_data["Cloudiness"]==0
ideal_data=weather_data[cond1 & cond2 & cond3 & cond4]
#drop rows with one or more null value
ideal_data=ideal_data.dropna()
ideal_data

,City,Country,Date,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed
2,Flinders,AU,1605322800,-34.5833,150.8552,74.25,57,0,7.29
59,Nelson,NZ,1605322800,-41.2833,173.2833,71.11,43,0,3.96
62,Raja,SS,1605322800,8.4596,25.6780,71.76,32,0,4.61
74,Gwadar,PK,1605322800,25.1216,62.3254,73.20,61,0,7.29
115,Kismayo,SO,1605322800,-0.3582,42.5454,78.94,78,0,7.25
134,Mae Chan,TH,1605322800,20.1467,99.8526,77.63,67,0,4.32
147,Formosa,AR,1605322800,-26.1775,-58.1781,76.77,53,0,4.41
153,Djougou,BJ,1605322800,9.7085,1.6660,72.63,39,0,5.48
225,Sheikh Dannun,IL,1605322800,32.9943,35.1479,70.14,45,0,8.34
229,Makkah al Mukarramah,SA,1605322800,21.4267,39.8261,74.62,63,0,3.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
#rename ideal_data to Hotel_df and add hotel name column
hotel_df=ideal_data
hotel_df["Hotel Name"]=''
hotel_df.head()

,City,Country,Date,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
2,Flinders,AU,1605322800,-34.5833,150.8552,74.25,57,0,7.29,
59,Nelson,NZ,1605322800,-41.2833,173.2833,71.11,43,0,3.96,
62,Raja,SS,1605322800,8.4596,25.6780,71.76,32,0,4.61,
74,Gwadar,PK,1605322800,25.1216,62.3254,73.20,61,0,7.29,
115,Kismayo,SO,1605322800,-0.3582,42.5454,78.94,78,0,7.25,


In [67]:
#set parameters to search for hotels
params= {"type":"lodging",
    "key": g_key,
    "radius": 5000 }
for index, rows in hotel_df.iterrows():
    base_url=base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat=hotel_df["Lat"]
    lng=hotel_df["Lng"]
    params['location']=f'{lat},{lng}'
    hotels=requests.get(base_url,params=params)
    hotels_json=hotels.json()
    try:
        hotel_df["Hotel Name"]=hotels_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Cannot find hotel...skipping")

Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping


In [64]:
params= {"type":"lodging",
    "key": g_key,
    "radius": 5000 }
base_url=base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
lat=hotel_df["Lat"]
lng=hotel_df["Lng"]
params['location']=f'{lat},{lng}'
requests.get(base_url,params=params).json()

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
